In [13]:
from PIL import ImageChops
import math, operator
from PIL import Image
from functools import reduce
import imagehash

def rmsdiff(im1, im2):
    "Calculate the root-mean-square difference between two images"

    h = ImageChops.difference(im1, im2).histogram()

    # calculate rms
    return math.sqrt(reduce(operator.add,
        map(lambda h, i: h*(i**2), h, range(256))
    ) / (float(im1.size[0]) * im1.size[1]))

def rmsdiff2(img1, img2):
    return imagehash.average_hash(img1) - imagehash.average_hash(img2)

def compare_images(img1, img2):
    # normalize to compensate for exposure difference, this may be unnecessary
    # consider disabling it
    img1 = normalize(img1)
    img2 = normalize(img2)
    # calculate the difference and its norms
    diff = img1 - img2  # elementwise for scipy arrays
    m_norm = sum(abs(diff))  # Manhattan norm
    z_norm = norm(diff.ravel(), 0)  # Zero norm
    return (m_norm, z_norm)


def equal(im1, im2):
    return ImageChops.difference(im1, im2).getbbox() is None

import sys

# from scipy.misc import imread
from scipy.linalg import norm
from scipy import sum, average
import imageio


def compare_images(img1, img2):
    # normalize to compensate for exposure difference, this may be unnecessary
    # consider disabling it
    img1 = normalize(img1)
    img2 = normalize(img2)
    # calculate the difference and its norms
    diff = img1 - img2  # elementwise for scipy arrays
    m_norm = sum(abs(diff))  # Manhattan norm
    z_norm = norm(diff.ravel(), 0)  # Zero norm
    return (m_norm, z_norm)

def to_grayscale(arr):
    "If arr is a color image (3D array), convert it to grayscale (2D array)."
    if len(arr.shape) == 3:
        return average(arr, -1)  # average over the last axis (color channels)
    else:
        return arr
    
def normalize(arr):
    rng = arr.max()-arr.min()
    amin = arr.min()
    return (arr-amin)*255/rng

def compare_main(file1, file2):
    img1 = to_grayscale(imageio.imread(file1).astype(float))
    img2 = to_grayscale(imageio.imread(file2).astype(float))
    # compare
    n_m, n_0 = compare_images(img1, img2)
    
    return n_m, n_0, img1, img2

In [15]:
file1 = "/home/dzanardo/github/apartamento/images_compare/2449863607/14.jpg"
file2 = "/home/dzanardo/github/apartamento/images_compare/2454100925/7.jpg"

im2 = Image.open(file1)
im1 = Image.open(file2)

print(rmsdiff(im2, im1))
print(rmsdiff2(im2, im1))

# read images as 2D arrays (convert to grayscale for simplicity)
n_m, n_0, img1, img2 = compare_main(file2, file1)
print("Manhattan norm:", n_m, "/ per pixel:", n_m/img1.size)
print("Zero norm:", n_0, "/ per pixel:", n_0*1.0/img1.size)

32.607018245906644
8
Manhattan norm: 2070915.2604712036 / per pixel: 23.968926625824114
Zero norm: 81797.0 / per pixel: 0.9467245370370371


/home/dzanardo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: DeprecationWarning: scipy.average is deprecated and will be removed in SciPy 2.0.0, use numpy.average instead
/home/dzanardo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: DeprecationWarning: scipy.sum is deprecated and will be removed in SciPy 2.0.0, use numpy.sum instead


In [18]:
import cv2
import numpy as np

file1 = "21.jpg"
file2 = "/home/dzanardo/github/apartamento/images_compare/2454100925/7.jpg"

img = cv2.imread(file1)

alpha = 2.0
beta = -160

new = alpha * img + beta
new = np.clip(new, 0, 255).astype(np.uint8)

cv2.imwrite("cleaned.png", new)

True